In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import plotly.express as px

project_folder = '/content/drive/MyDrive/ProjectTCNER'
data_folder = os.path.join(project_folder, "data")
trainset_file = os.path.join(data_folder, "DBLPTrainset.txt")
testset_file = os.path.join(data_folder, "DBLPTestset.txt")
ground_truth_file = os.path.join(data_folder, "DBLPTestGroundTruth.txt")

In [3]:
dataset_train = pd.read_table(os.path.join(trainset_file), sep="\t", header=None, names=['y', 'X'], index_col=0)
dataset_train

,y,X
0,ISCAS,Scalable Serial-parallel Multiplier over GF(2m...
1,SIGGRAPH,Plenoptic sampling.
2,ISCAS,Sensitivity and uniformity of a 0.18micrometer...
3,WWW,A survey of web archive search architectures.
4,ISCAS,Understanding dynamic behavior of mm-wave CML ...
...,...,...
21638,ISCAS,Decoding a Family of Dense Codes using the Sum...
21639,VLDB,CoHadoop: Flexible Data Placement and Its Expl...
21640,ISCAS,Full system simulation with QEMU: An approach ...
21641,INFOCOM,Localization in non-localizable sensor and ad-...


In [4]:
dataset_test = pd.read_table(os.path.join(testset_file), sep="\t", header=None, names=['X'], index_col=0)
dataset_test

,X
0,Fast recursive adaptation for nonlinear filters.
1,High-Throughput Data Compressor Designs Using ...
2,Functional Verification of ECL Circuits Includ...
3,Efficient network generation under general pre...
4,Creating the earth as a backdrop in <i>Gravity...
...,...
3368,GEM: A Geometric Algorithm for Scheduling.
3369,On the geographic patterns of a large-scale mo...
3370,NScale: Neighborhood-centric Analytics on Larg...
3371,Sufficient Conditions for Finding Multiple Ope...


In [5]:
dataset_gt = pd.read_table(os.path.join(ground_truth_file), sep="\t", header=None, names=['y'], index_col=0)
dataset_gt

,y
0,ISCAS
1,ISCAS
2,ISCAS
3,WWW
4,SIGGRAPH
...,...
3368,ISCAS
3369,INFOCOM
3370,VLDB
3371,ISCAS


In [6]:
dataset_test = pd.merge(dataset_test, dataset_gt, left_index=True, right_index=True)
dataset_test

,X,y
0,Fast recursive adaptation for nonlinear filters.,ISCAS
1,High-Throughput Data Compressor Designs Using ...,ISCAS
2,Functional Verification of ECL Circuits Includ...,ISCAS
3,Efficient network generation under general pre...,WWW
4,Creating the earth as a backdrop in <i>Gravity...,SIGGRAPH
...,...,...
3368,GEM: A Geometric Algorithm for Scheduling.,ISCAS
3369,On the geographic patterns of a large-scale mo...,INFOCOM
3370,NScale: Neighborhood-centric Analytics on Larg...,VLDB
3371,Sufficient Conditions for Finding Multiple Ope...,ISCAS


In [7]:
X_train = dataset_train['X']
y_train = dataset_train['y']

In [8]:
# Test set
y_test = pd.read_table(os.path.join(ground_truth_file), sep="\t", header=None, names=['y'],
                              index_col=0)['y']
X_test = pd.read_table(os.path.join(testset_file), sep="\t", header=None, names=['X'],
                       index_col=0)['X']

BERT

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(5, activation='softmax', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs = [l])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=10, batch_size = 32)
model.evaluate(X_test, y_test)

y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
print(y_predicted)